**박보현**

***Image Classification ***

- Cifar10 데이터셋에 CNN 적용


CIFAR-10 Dataset
- Tr: 60,000 / Ts: 10,000
- 각 샘플은 32x32 컬러 이미지
- 클래스: 10개

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader

from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# parameters
learning_rate = 0.001
epochs = 100
batch_size = 100
drop_out = 0.3

**데이터 준비**

In [ ]:
# CIFAR-10 dataset
training_data = datasets.CIFAR10(root='./data',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

test_data = datasets.CIFAR10(root='./data',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
print(training_data)
print(test_data)

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: ToTensor()
Dataset CIFAR10
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: ToTensor()


In [ ]:
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

**신경망 구성**

In [ ]:
class CNN(nn.Module):

  def __init__(self):
    super(CNN, self).__init__()
    #첫번째 layer 생성
    self.layer1 = nn.Sequential(
        nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1), #CIFAR-10 dataset은 RGB 이미지로 구성되어 있으므로 입력 이미지의 채널 수는 3
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)) #최초 32x32 -> max pooling 16x16

    #두번째 layer 생성
    self.layer2 = nn.Sequential(
        nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)) #16x16 -> max pooling 8x8

    #fully connected layer 생성
    self.fc = nn.Linear(8*8*64, 10, bias=True)
    nn.init.xavier_uniform_(self.fc.weight)

  #모델 설계 후 데이터셋이 layer들을 통과할 수 있게 함
  def forward(self,x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = out.view(out.size(0), -1)
    out = self.fc(out)
    return out

In [ ]:
class CNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        #첫번째 layer 생성
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 두번째층
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 전결합층 8x8x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(8 * 8 * 64, 10, bias=True)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = torch.nn.Flatten()(out)   # 전결합층을 위해서 Flatten
        out = self.fc(out)
        return out

In [ ]:
model = CNN().to(device)

print(model)

CNN(
  (layer1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=4096, out_features=10, bias=True)
)


In [ ]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

print(criterion)
print(optimizer)

CrossEntropyLoss()
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)


**학습**

In [ ]:
total_batch = len(train_dataloader)
print('Learning started. It takes sometime.')

for epoch in range(epochs):
  avg_loss = 0                                  # 비용 초기값

  for i, data in enumerate(train_dataloader):
    inputs, labels = data

    inputs = inputs.to(device)                  # inputs -> 디바이스로 옮김
    labels = labels.to(device)                  # labels -> 디바이스로 옮김

    # 변화도(Gradient) 매개변수를 0으로 만듦
    optimizer.zero_grad()

    # 순전파 + 역전파 + 최적화
    outputs = model(inputs)                     # 가설 설정
    loss = criterion(outputs, labels)           # 로스 설정
    loss.backward()                             # 역전파
    optimizer.step()                            # 학습을 통해 계산한 weight, bias, gradient를 최적화 함수(optimizer)에 반영

    avg_loss += loss / total_batch

  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_loss))

print('Learning Finished!')

Learning started. It takes sometime.
[Epoch:    1] cost = 1.53438878
[Epoch:    2] cost = 1.23419082
[Epoch:    3] cost = 1.09338188
[Epoch:    4] cost = 1.00162578
[Epoch:    5] cost = 0.939672589
[Epoch:    6] cost = 0.890537858
[Epoch:    7] cost = 0.853082597
[Epoch:    8] cost = 0.819725394
[Epoch:    9] cost = 0.789833724
[Epoch:   10] cost = 0.763511896
[Epoch:   11] cost = 0.737067044
[Epoch:   12] cost = 0.712428927
[Epoch:   13] cost = 0.691352189
[Epoch:   14] cost = 0.66801393
[Epoch:   15] cost = 0.648395538
[Epoch:   16] cost = 0.632365584
[Epoch:   17] cost = 0.616243422
[Epoch:   18] cost = 0.607901752
[Epoch:   19] cost = 0.589334667
[Epoch:   20] cost = 0.577199101
[Epoch:   21] cost = 0.561478376
[Epoch:   22] cost = 0.548938572
[Epoch:   23] cost = 0.534178376
[Epoch:   24] cost = 0.523932636
[Epoch:   25] cost = 0.511346042
[Epoch:   26] cost = 0.500216246
[Epoch:   27] cost = 0.49210307
[Epoch:   28] cost = 0.4851408
[Epoch:   29] cost = 0.474700928
[Epoch:   30] 

**테스트**

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for data in test_dataloader:
        images, labels = data

        images = images.to(device)
        labels = labels.to(device)

        # 신경망에 이미지를 통과시켜 출력을 계산
        outputs = model(images)
        # 가장 높은 값(energy)를 갖는 분류(class)를 정답으로 선택
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 66 %
